### Introduction

Azure Machine Learning is a platform for operating machine learning workloads in the cloud.

Azure Machine Learning belongs to the Microsoft Azure platform and enable the users to manage:
- Scalable on-demand compute for machine learning workloads.
- Data storage and connectivity from a wide range of sources.
- Machine Learning workflow orchestration to automate model training, deployment and management
processes.
- Model registration and management.
- Metrics and monitoring for training experiments, datasets and published services.
- Model deployment for real-time and batch inference.

### Workspace

A _workspace_ is a context for the experiments, data, compute target and other assets associated with a machine learning workloads. It defines the boundary for machine learning assets. It includes:
- Compute targets
- Data
- Notebooks containing sharing code and documentation.
- Experiments
- Pipelines that define orchestrated multi-step processes.
- Models

A workspace can be created by:
- The Azure portal
- Using Azure Machine Learning Python SDK to run some code that creates a workspace.

In [2]:
from azureml.core import Workspace

In [ ]:
# Create a new workspace.
ws = Workspace.create(name = 'aml-workspace',
                      subscription_id = '#####',
                      resource_group = '####',
                      create_resource_group = False,
                      location = '####')


To create a compute instance, you should go to Azure ML studio and choose between:
- _Compute instances_: development workstations that ds can use to work with data and models.
- _Compute clusters_: scalable clusters of VMs.
- _Inference clusters_: deployment targets for predictive services that use your trained model.
- _Attached compute_: links to other Azure compute resources, such as Virtual Machines or Azure Databricks clusters.